# Микродиахроническое исследование русских приставок методами дистрибутивной семантики
## Автор: Елизавета Клыкова, БКЛ181
### Получение и анализ грамматических профилей глаголов
1. Считать подготовленные pickle-файлы с грамматической информацией о токенах
2. Составить представления глаголов на основе их токенов

#### Импорт модулей

In [1]:
# %load_ext pycodestyle_magic
# %pycodestyle_on

In [2]:
import re
import pickle
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from collections import Counter
from statistics import mean
from scipy.spatial.distance import cityblock

#### Получение списка глаголов

In [3]:
pref_df = pd.read_csv('prefixes_and_lemmas_dropna.tsv', sep='\t')
pref_df

,prefix,lemma,abs_freq,abs_freq0,abs_freq1,abs_freq2,freq,freq0,freq1,freq2
0,бе[зс],бездействовать,541,155,255,131,2.194,2.147,2.741,1.609
1,бе[зс],бездельничать,308,41,169,98,1.249,0.568,1.817,1.204
2,бе[зс],безмолвствовать,726,361,191,174,2.944,5.000,2.053,2.138
3,бе[зс],безобразить,149,99,43,7,0.604,1.371,0.462,0.086
4,бе[зс],безобразничать,306,110,136,60,1.241,1.524,1.462,0.737
...,...,...,...,...,...,...,...,...,...,...
8429,у,ущемляться,62,8,16,38,0.251,0.111,0.172,0.467
8430,у,ущипывать,510,147,222,141,2.068,2.036,2.386,1.732
8431,у,уязвлять,1305,606,411,288,5.291,8.393,4.418,3.538
8432,у,уяснять,1803,677,740,386,7.311,9.377,7.954,4.742


In [4]:
# list of dicts
pref_dict = pref_df.to_dict(orient='records')

In [5]:
verbs_to_search = sorted(set([v['lemma'] for v in pref_dict]))

#### Получение грамматических профилей
Открываем ранее полученные файлы:

In [6]:
with open('presov_verb_dict.pickle', 'rb') as f:
    presov_verb_dict = pickle.load(f)

In [7]:
with open('sov_verb_dict.pickle', 'rb') as f:
    sov_verb_dict = pickle.load(f)

In [8]:
with open('postsov_verb_dict.pickle', 'rb') as f:
    postsov_verb_dict = pickle.load(f)

(Giulianelli et al. 2022):
- Tense : {Past 42, Pres 51}
- VerbForm : {Part 68, Fin 25, Inf 9}
- Mood : {Ind 25}
- Voice : {Pass 17}

Для нашего датасета:
- tense: pres, inpr, past
- verbform: ger, part, inf, fin
- mood: ind, imp
- number: sg, pl
- person: 1p, 2p, 3p
- gender: m, f, n
- voice: act, pass

Другое:
- вид: perf, imperf
- переходность: trans, intrans
- падеж: есть только у причастий
- форма: краткая/полная -- есть только у причастий
- одушевленность: вообще что-то странное

In [9]:
# full = {'tense': {'непрош': 0, 'прош': 0},
#         'vform': {'прич': 0, 'деепр': 0, 'инф': 0, 'фин': 0},
#         'mood': {'пов': 0, 'инд': 0},
#         'numb': {'ед': 0, 'мн': 0},
#         'pers': {'1-л': 0, '2-л': 0, '3-л': 0},
#         'gend': {'муж': 0, 'жен': 0, 'сред': 0},
#         'voice': {'действ': 0, 'страд': 0}
#         }

In [10]:
# presov_gram_dict = {}
# gram_list = set(['непрош', 'прош', 'прич', 'деепр', 'инф', 'фин', 'пов', 'изъяв', 'ед', 'мн', '1-л', '2-л', '3-л', 'муж', 'жен', 'сред', 'действ', 'страд'])
# # grams_to_ignore = set(['полн', 'кр', 'им', 'вин', 'дат', 'твор', 'пр', 'од', 'неод'])

# for verb in tqdm(presov_verb_dict):
#     presov_gram_dict[verb] = {'непрош': 0, 'прош': 0,
#                               'прич': 0, 'деепр': 0, 'инф': 0, 'фин': 0,
#                               'пов': 0, 'изъяв': 0,
#                               'ед': 0, 'мн': 0,
#                               '1-л': 0, '2-л': 0, '3-л': 0,
#                               'муж': 0, 'жен': 0, 'сред': 0,
#                               'действ': 0, 'страд': 0}
#     for token in presov_verb_dict[verb]:
#         gram_info = re.sub('^V[=,]', '', token['gram'])
#         # убрать одушевленность
#         gram_info = re.sub('сов|несов|нп|пе|устар|обсц|разг|полн|кр|им|вин|род|дат|твор|од|неод', '', gram_info)
#         gram_info = re.sub(',+', ',', gram_info).strip('(,=').split(',')
#         for gram in gram_info:
#             if gram in gram_list:
#                 presov_gram_dict[verb][gram] += 1
#         # не нужно ли это убрать?
#         if 'прич' not in gram_info and 'деепр' not in gram_info and 'инф' not in gram_info:
#             presov_gram_dict[verb]['фин'] += 1

In [11]:
def get_gram_profiles(verb_dict, all_grams=set()):

    gram_dict = {}

    for verb in tqdm(verb_dict):
        gram_list = []

        for token in verb_dict[verb]:
            gram_info = re.sub('^V[=,]', '', token['gram'])
            gram_info = re.sub('сов|несов|нп|пе|устар|обсц|разг|полн|кр|им|вин|род|дат|твор|од|неод',
                               '', gram_info)
            gram_info = re.sub(',+', ',', gram_info).strip('(,=').split(',')
            gram_info = sorted(set([gram for gram in gram_info if gram != 'пр']))
            gram = '_'.join(gram_info)
            gram_list.append(gram)
            all_grams.add(gram)

        gram_counter = list(Counter(gram_list).items())

        # нормализация
        tok_num = len(verb_dict[verb])
        for i, gram in enumerate(gram_counter):
            norm_gram = gram[1] / tok_num
            gram_counter[i] = (gram[0], norm_gram)

        # сохраняем грамматический профиль глагола
        gram_dict[verb] = gram_counter

    return gram_dict, all_grams

In [12]:
presov_gram_dict, all_grams = get_gram_profiles(presov_verb_dict)

  0%|          | 0/8434 [00:00<?, ?it/s]

In [13]:
presov_gram_dict['беспокоиться']

[('инф', 0.2601544142329641),
 ('2-л_ед_пов', 0.12487411883182276),
 ('2-л_мн_пов', 0.27257468949311847),
 ('1-л_ед_изъяв_непрош', 0.030211480362537766),
 ('ед_жен_изъяв_прош', 0.03893924135615979),
 ('2-л_изъяв_мн_непрош', 0.0433031218529708),
 ('изъяв_мн_прош', 0.03726082578046324),
 ('3-л_ед_изъяв_непрош', 0.06411547499160793),
 ('ед_изъяв_муж_прош', 0.07452165156092648),
 ('2-л_ед_изъяв_непрош', 0.010406176569318564),
 ('действ_мн_непрош_прич', 0.0016784155756965425),
 ('3-л_изъяв_мн_непрош', 0.018798254447801276),
 ('деепр_непрош', 0.013763007720711649),
 ('действ_ед_муж_непрош_прич', 0.002014098690835851),
 ('действ_ед_муж_прич_прош', 0.000671366230278617),
 ('1-л_изъяв_мн_непрош', 0.0030211480362537764),
 ('деепр_прош', 0.0003356831151393085),
 ('действ_ед_жен_прич_прош', 0.000671366230278617),
 ('действ_мн_прич_прош', 0.0003356831151393085),
 ('ед_изъяв_прош_сред', 0.0010070493454179255),
 ('действ_ед_жен_непрош_прич', 0.001342732460557234)]

In [14]:
sov_gram_dict, all_grams = get_gram_profiles(sov_verb_dict, all_grams)

  0%|          | 0/8434 [00:00<?, ?it/s]

In [15]:
postsov_gram_dict, all_grams = get_gram_profiles(postsov_verb_dict, all_grams)

  0%|          | 0/8434 [00:00<?, ?it/s]

#### Получаем векторы глаголов
Сначала нужно убедиться, что словарь с каждым профилем содержит все возможные ключи.

In [16]:
def unify_gram_keys(gram_dict, all_grams):

    for verb in gram_dict:
        verb_forms = set([form[0] for form in gram_dict[verb]])
        missing_grams = all_grams - verb_forms
        for gram in missing_grams:
            gram_dict[verb].append((gram, 0))
        gram_dict[verb] = sorted(gram_dict[verb], key=lambda x: x[0])

    return gram_dict

In [17]:
presov_gram_dict = unify_gram_keys(presov_gram_dict, all_grams)
sov_gram_dict = unify_gram_keys(sov_gram_dict, all_grams)
postsov_gram_dict = unify_gram_keys(postsov_gram_dict, all_grams)

Теперь переводим словари в векторы.

In [18]:
presov_vect_dict = {verb: np.array([gram[1] for gram in presov_gram_dict[verb]])
                    for verb in presov_gram_dict}

In [19]:
len(presov_verb_dict['беспокоиться'])

2979

In [20]:
presov_vect_dict['беспокоиться']

array([0.03021148, 0.00302115, 0.        , 0.01040618, 0.12487412,
       0.04330312, 0.27257469, 0.06411547, 0.01879825, 0.01376301,
       0.00033568, 0.00134273, 0.00067137, 0.0020141 , 0.00067137,
       0.        , 0.        , 0.00167842, 0.00033568, 0.03893924,
       0.        , 0.        , 0.07452165, 0.00100705, 0.        ,
       0.        , 0.        , 0.        , 0.03726083, 0.        ,
       0.        , 0.26015441, 0.        , 0.        ])

In [21]:
sov_vect_dict = {verb: np.array([gram[1] for gram in sov_gram_dict[verb]])
                 for verb in sov_gram_dict}
postsov_vect_dict = {verb: np.array([gram[1] for gram in postsov_gram_dict[verb]])
                     for verb in postsov_gram_dict}

In [22]:
for verb in pref_dict:
    lemma = verb['lemma']
    verb['gram_presov'] = presov_vect_dict[lemma]
    verb['gram_sov'] = sov_vect_dict[lemma]
    verb['gram_postsov'] = postsov_vect_dict[lemma]

#### Рассчитываем степень изменения между периодами

In [23]:
for verb in tqdm(pref_dict):
    verb['gram_pre_to_sov'] = round(cityblock(verb['gram_presov'], verb['gram_sov']), 3)
    verb['gram_sov_to_post'] = round(cityblock(verb['gram_sov'], verb['gram_postsov']), 3)
    verb['gram_pre_to_post'] = round(cityblock(verb['gram_presov'], verb['gram_postsov']), 3)

  0%|          | 0/8434 [00:00<?, ?it/s]

#### Ранжируем по степени изменения в каждом периоде

In [31]:
verbs_change_pre_to_sov = [{'lemma': verb['lemma'],
                            'abs_freq0': verb['abs_freq0'],
                            'abs_freq1': verb['abs_freq1'],
                            'abs_freq2': verb['abs_freq2']}
                           for verb in sorted(pref_dict,
                                              key=lambda x:x['gram_pre_to_sov'],
                                              reverse=True)]
verbs_change_sov_to_post = [{'lemma': verb['lemma'],
                            'abs_freq0': verb['abs_freq0'],
                            'abs_freq1': verb['abs_freq1'],
                            'abs_freq2': verb['abs_freq2']}
                           for verb in sorted(pref_dict,
                                              key=lambda x:x['gram_sov_to_post'],
                                              reverse=True)]
verbs_change_pre_to_post = [{'lemma': verb['lemma'],
                            'abs_freq0': verb['abs_freq0'],
                            'abs_freq1': verb['abs_freq1'],
                            'abs_freq2': verb['abs_freq2']}
                           for verb in sorted(pref_dict,
                                              key=lambda x:x['gram_pre_to_post'],
                                              reverse=True)]

In [32]:
verbs_change_pre_to_sov = [verb['lemma'] for verb in verbs_change_pre_to_sov
                           if verb['abs_freq0'] > 100 and verb['abs_freq1'] > 100 and verb['abs_freq2'] > 100][0:200]
verbs_change_sov_to_post = [verb['lemma'] for verb in verbs_change_sov_to_post
                            if verb['abs_freq0'] > 100 and verb['abs_freq1'] > 100 and verb['abs_freq2'] > 100][0:200]
verbs_change_pre_to_post = [verb['lemma'] for verb in verbs_change_pre_to_post
                            if verb['abs_freq0'] > 100 and verb['abs_freq1'] > 100 and verb['abs_freq2'] > 100][0:200]

#### Выбираем топ-10 наиболее изменившихся глаголов для дальнейшего анализа в ELMo

In [33]:
verbs_for_elmo = []
for verb in verbs_change_pre_to_sov:
    if verb in verbs_change_sov_to_post and verb in verbs_change_pre_to_post:
        verbs_for_elmo.append(verb)
    if len(verbs_for_elmo) == 10:
        break

In [34]:
print(verbs_for_elmo)

['заметать', 'постановлять', 'изготавливаться', 'наследовать', 'претерпеть', 'взводить', 'засылать', 'дозволять', 'устрашать', 'округлять']


#### Рассчитываем изменения в приставках

In [29]:
prefixes = sorted(list(set([verb['prefix'] for verb in pref_dict])))

In [30]:
gram_tok_dict = {prefix: [] for prefix in prefixes}

for verb in pref_dict:
    gram_tok_dict[verb['prefix']].append(verb)

In [31]:
for prefix in gram_tok_dict:
    verbs = gram_tok_dict[prefix]
    pre_to_sov = [(verb['lemma'], verb['gram_pre_to_sov'])
                  for verb in sorted(verbs,
                                     key=lambda x: x['gram_pre_to_sov'],
                                     reverse=True)]
    sov_to_post = [(verb['lemma'], verb['gram_sov_to_post'])
                   for verb in sorted(verbs,
                                      key=lambda x: x['gram_sov_to_post'],
                                      reverse=True)]
    pre_to_post = [(verb['lemma'], verb['gram_pre_to_post'])
                   for verb in sorted(verbs,
                                      key=lambda x: x['gram_pre_to_post'],
                                      reverse=True)]
    gram_tok_dict[prefix] = {'pre_to_sov_mean': round(mean([v[1] for v in pre_to_sov]), 3),
                             'sov_to_post_mean': round(mean([v[1] for v in sov_to_post]), 3),
                             'pre_to_post_mean': round(mean([v[1] for v in pre_to_post]), 3),
                             'pre_to_sov_ranged': pre_to_sov,
                             'sov_to_post_ranged': sov_to_post,
                             'pre_to_post_ranged': pre_to_post}

In [32]:
gram_tok_dict['полу']['sov_to_post_mean']

0.473

#### Ранжируем приставки по степени изменения

In [33]:
prefs_pre_to_sov_ranged = [(pref,
                            len(gram_tok_dict[pref]['pre_to_sov_ranged']),
                            gram_tok_dict[pref]['pre_to_sov_mean'])
                           for pref in sorted(gram_tok_dict,
                                              key=lambda x: gram_tok_dict[x]['pre_to_sov_mean'],
                                              reverse=True)]

In [34]:
prefs_sov_to_post_ranged = [(pref,
                             len(gram_tok_dict[pref]['sov_to_post_ranged']),
                             gram_tok_dict[pref]['sov_to_post_mean'])
                            for pref in sorted(gram_tok_dict,
                                               key=lambda x: gram_tok_dict[x]['sov_to_post_mean'],
                                               reverse=True)]

In [35]:
prefs_pre_to_post_ranged = [(pref,
                             len(gram_tok_dict[pref]['pre_to_post_ranged']),
                             gram_tok_dict[pref]['pre_to_post_mean'])
                            for pref in sorted(gram_tok_dict,
                                               key=lambda x: gram_tok_dict[x]['pre_to_post_mean'],
                                               reverse=True)]

In [36]:
pref_ranged_df = pd.DataFrame(
    {'gram_pre_to_sov': prefs_pre_to_sov_ranged,
     'gram_sov_to_post': prefs_sov_to_post_ranged,
     'gram_pre_to_post': prefs_pre_to_post_ranged,
     })

In [37]:
pref_ranged_df

,gram_pre_to_sov,gram_sov_to_post,gram_pre_to_post
0,"(недо, 17, 0.757)","(недо, 17, 0.643)","(недо, 17, 0.901)"
1,"(обе[зс], 27, 0.621)","(обе[зс], 27, 0.582)","(обе[зс], 27, 0.715)"
2,"(над, 17, 0.605)","(бе[зс], 11, 0.574)","(над, 17, 0.689)"
3,"(пере, 356, 0.563)","(ни[зс], 10, 0.539)","(на, 523, 0.657)"
4,"(на, 523, 0.553)","(над, 17, 0.521)","(ни[зс], 10, 0.657)"
5,"(про, 483, 0.544)","(во[зс], 122, 0.506)","(пере, 356, 0.657)"
6,"(во?, 137, 0.54)","(пере, 356, 0.502)","(про, 483, 0.638)"
7,"(до, 153, 0.538)","(на, 523, 0.49)","(во?, 137, 0.637)"
8,"(от, 393, 0.534)","(до, 153, 0.482)","(бе[зс], 11, 0.625)"
9,"(вы, 444, 0.525)","(о, 815, 0.479)","(во[зс], 122, 0.625)"


In [38]:
ranged_dict = pref_ranged_df.to_dict(orient='records')

In [39]:
# приводим к стандартному виду
convt = {'бе[зс]': 'без-',
         'в[зс]': 'вз-',
         'во?': 'в-',
         'во[зс]': 'воз-',
         'вы': 'вы-',
         'до': 'до-',
         'за': 'за-',
         'и[зс]': 'из-',
         'на': 'на-',
         'над': 'над-',
         'недо': 'недо-',
         'ни[зс]': 'низ-',
         'о': 'о-',
         'обе[зс]': 'обез-',
         'от': 'от-',
         'пере': 'пере-',
         'по': 'по-',
         'под': 'под-',
         'полу': 'полу-',
         'пре': 'пре-',
         'пред': 'пред-',
         'при': 'при-',
         'про': 'про-',
         'ра[зс]': 'раз-',
         'с': 'с-',
         'у': 'у-'}

In [40]:
for row in ranged_dict:
    for col in row:
        row[col] = convt[row[col][0]]

In [41]:
clean_ranged_df = pd.DataFrame(ranged_dict)
clean_ranged_df

,gram_pre_to_sov,gram_sov_to_post,gram_pre_to_post
0,недо-,недо-,недо-
1,обез-,обез-,обез-
2,над-,без-,над-
3,пере-,низ-,на-
4,на-,над-,низ-
5,про-,воз-,пере-
6,в-,пере-,про-
7,до-,на-,в-
8,от-,до-,без-
9,вы-,о-,воз-
